In [25]:
import pandas as pd
import numpy as np
from datetime import datetime
import io
import sagemaker.amazon.common as smac

import boto3
from sagemaker import get_execution_role
import sagemaker
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [20]:
#from helper import myClass1

In [55]:
from sagemaker import get_execution_role

In [23]:
class myClass1:
    def get_s3_object(self,bucket,key):
        s3 = boto3.client('s3')
        obj = s3.get_object(Bucket=bucket, Key=key)
        df = pd.read_csv(io.BytesIO(obj['Body'].read()))
        return df
    
    def upload_to_s3(self,bucket,key,file_path):
        boto3.Session().resource('s3').Bucket(bucket).Object(key).upload_file(file_path)

awsObjects = myClass1()

In [56]:
role = get_execution_role()

In [24]:
bucket = 'ars-mlops-projects'
key = 'BankPersonalLoanModelling/data/processed_data.csv'
df = awsObjects.get_s3_object(bucket,key)
df.head(3)

,Unnamed: 0,ID,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard,Personal Loan
0,0,1,25,1,49,4,1.6,1,0,1,0,0,0,0
1,1,2,45,19,34,3,1.5,1,0,1,0,0,0,0
2,2,3,39,15,11,1,1.0,1,0,0,0,0,0,0


In [27]:
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [29]:
df.drop(labels=['ID'],axis=1,inplace=True)
df.head(3)

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard,Personal Loan
0,25,1,49,4,1.6,1,0,1,0,0,0,0
1,45,19,34,3,1.5,1,0,1,0,0,0,0
2,39,15,11,1,1.0,1,0,0,0,0,0,0


# Segregate training and Validation Data

In [30]:
x_train, x_test , y_train, y_test = train_test_split(df.drop('Personal Loan',axis=1), df['Personal Loan'], test_size = 0.33,random_state=42)

In [38]:
df_new = df
df_train = x_train
df_train['Personal Loan'] = y_train
df_val = x_test
df_val['Personal Loan'] = y_test

In [41]:
pd.concat([df_train['Personal Loan'],df_train.drop(['Personal Loan'],axis=1)],axis=1).to_csv('data/train.csv',index=False,header=False)

In [44]:
pd.concat([df_val['Personal Loan'],df_val.drop(['Personal Loan'],axis=1)],axis=1).to_csv('data/validation.csv',index=False,header=False)

In [50]:
train_key = 'BankPersonalLoanModelling/data/train/train.csv'
awsObjects.upload_to_s3(bucket,train_key,'data/train.csv')
val_key = 'BankPersonalLoanModelling/data/validation/validation.csv'
awsObjects.upload_to_s3(bucket,val_key,'data/validation.csv')

In [45]:
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://ars-mlops-projects/BankPersonalLoanModelling/data/train.csv'.format(bucket), content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data='s3://ars-mlops-projects/BankPersonalLoanModelling/data/validation.csv'.format(bucket), content_type='csv')

# Creating and Training XGBoost

In [53]:
from sagemaker import image_uris
container = image_uris.retrieve('xgboost',boto3.Session().region_name,'1')

In [51]:
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://ars-mlops-projects/BankPersonalLoanModelling/data/train'.format(bucket), content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data='s3://ars-mlops-projects/BankPersonalLoanModelling/data/validation/validation.csv'.format(bucket), content_type='csv')


In [52]:
job_name = 'BankPersonalLoan-xgboost-job-{}'.format(datetime.now().strftime("%Y%m%d%H%M%S"))
print('Here is the job name {}'.format(job_name))
output_location = 's3://ars-mlops-projects/BankPersonalLoanModelling/output/'.format(bucket)

Here is the job name BankPersonalLoan-xgboost-job-20220422025830


In [57]:
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(
                        container,
                        role,
                        instance_count=1, 
                        instance_type='ml.m4.xlarge',
                        output_path=output_location,
                        sagemaker_session=sess
    )

In [58]:
xgb.set_hyperparameters(objective='multi:softmax',
                        num_class = 7,
                        num_round=100)
data_channels = {
    'train': s3_input_train,
    'validation' : s3_input_validation
}

xgb.fit(data_channels,job_name=job_name)

2022-04-22 03:00:09 Starting - Starting the training job...
2022-04-22 03:00:35 Starting - Preparing the instances for trainingProfilerReport-1650596409: InProgress
.........
2022-04-22 03:01:53 Downloading - Downloading input data...
2022-04-22 03:02:33 Training - Downloading the training image......
2022-04-22 03:03:34 Training - Training image download completed. Training in progress..Arguments: train
[2022-04-22:03:03:34:INFO] Running standalone xgboost training.
[2022-04-22:03:03:34:INFO] File size need to be processed in the node: 0.31mb. Available memory size in the node: 8489.25mb
[2022-04-22:03:03:34:INFO] Determined delimiter of CSV input is ','
[03:03:34] S3DistributionType set as FullyReplicated
[03:03:34] 6056x11 matrix with 66616 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2022-04-22:03:03:34:INFO] Determined delimiter of CSV input is ','
[03:03:34] S3DistributionType set as FullyReplicated
[03:03:34] 2984x11 matrix with 32824 entri